In [24]:
import pandas as pd
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet,  LassoLarsIC, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, XGBClassifier, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone, BaseEstimator, TransformerMixin, RegressorMixin # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import explained_variance_score, median_absolute_error, r2_score, mean_squared_error #To evaluate our model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression

os.chdir("../input")

In [10]:
def convert_int(x):
    try:
        return int(x)
    except ValueError:
        return int(x[:-1])

In [12]:
train = pd.concat([pd.read_csv("MRegularSeasonCompactResults.csv"),pd.read_csv("MNCAATourneyCompactResults.csv")])
seed_df = pd.read_csv("MNCAATourneySeeds.csv")
train = train.rename(columns={"WTeamID":"ATeamID","WScore":"AScore","LTeamID":"BTeamID","LScore":"BScore"})

In [14]:
split = [True if random.randint(0,1)==1 else False for x in range(len(train))]
train["split"] = split

switch_train = train[~train["split"]].drop(columns=["split"])
switch_train = switch_train.rename(columns={"BLoc":"ALoc","ATeamID":"BTeamID","AScore":"BScore","BTeamID":"ATeamID","BScore":"AScore"})

train = train[train["split"]].drop(columns=["split"])

train["y"] = 1
switch_train["y"] = 0

In [274]:
#switch_train = train #- OVERFIT

In [15]:
train = pd.concat([train,switch_train])
train = train.merge(seed_df, left_on=["ATeamID","Season"], right_on=["TeamID","Season"])
train = train.merge(seed_df, left_on=["BTeamID","Season"], right_on=["TeamID","Season"])
train = train.rename(columns={"Seed_x":"ASeed","Seed_y":"BSeed"}).drop(columns=["TeamID_x","TeamID_y"])
train.loc[:,["AZone","ASeed"]] = list(train.ASeed.apply(lambda x: [x[0],convert_int(x[1:])]))
train.loc[:,["BZone","BSeed"]] = list(train.BSeed.apply(lambda x: [x[0],convert_int(x[1:])]))

In [16]:
y_train = train["y"]
train = train[["Season","ATeamID","BTeamID","ASeed","BSeed"]] #,"AZone","BZone"


In [17]:
test = pd.read_csv("../input/MSampleSubmissionStage1.csv")
test_ID = test.ID
test = pd.DataFrame(list(test.ID.apply(lambda x: x.split("_"))), test_ID, columns=["Season","ATeamID","BTeamID"]).astype("int").reset_index()
test = test.merge(seed_df, left_on=["ATeamID","Season"], right_on=["TeamID","Season"])
test = test.merge(seed_df, left_on=["BTeamID","Season"], right_on=["TeamID","Season"])
test = test.rename(columns={"Seed_x":"ASeed","Seed_y":"BSeed"}).drop(columns=["TeamID_x","TeamID_y"])
test.loc[:,["AZone","ASeed"]] = list(test.ASeed.apply(lambda x: [x[0],convert_int(x[1:])]))
test.loc[:,["BZone","BSeed"]] = list(test.BSeed.apply(lambda x: [x[0],convert_int(x[1:])]))
test = test.set_index("ID").loc[test_ID]
test = test[["Season","ATeamID","BTeamID","ASeed","BSeed"]]




In [18]:
X = pd.concat([train,test])
X = pd.get_dummies(X)
train = X.iloc[:len(y_train)]
test = X.iloc[len(y_train):]
train, y_train = shuffle(train, y_train)

In [21]:
model=LGBMRegressor()

def rmse(X, y,model=model):
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    rmse = np.sqrt(-cross_val_score(model, X,y, cv= kfold, scoring='neg_mean_squared_error', n_jobs=1))
    return rmse

def r2(X, y,model=model):
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    r2 = cross_val_score(model, X, y, cv= kfold, scoring="r2", n_jobs=1)
    return r2

seed = 5
n_folds =5 

In [26]:
print("RMSE %f" % rmse(train,y_train).mean())
print("R2 %f" % r2(train,y_train).mean())
model.fit(train,y_train)
model.predict(test)

RMSE 0.461681
R2 0.145717


array([0.21281521, 0.08011287, 0.11666215, ..., 0.6145399 , 1.06036049,
       1.02219741])

In [178]:
import pickle
pkl_filename = "../output/lgbm.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [283]:
finalPred = model.predict(test)
finalPred = [min(max(x,0),1) for x in finalPred]

In [284]:
sub = pd.DataFrame()
sub['ID'] = test_ID
sub['Pred'] = finalPred
#sub.to_csv('../output/lgb_submission.csv',index=False)

In [285]:
from sklearn.metrics import mean_squared_error as MSE
predictions = sub["Pred"]
y_test = np.log1p(pd.read_csv("../output/lgb_submission.csv")["Pred"])
rmse = np.sqrt(MSE(y_test, predictions)) 
print("RMSE : % f" %(rmse)) 

RMSE :  0.214457
